Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logit

In [ ]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # L2 regularization
  regularizers = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss += 5e-4 * regularizers

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

## DNN

In [10]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases = tf.Variable(tf.zeros([hidden_size]))
    
  weights_hidden = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  out = tf.matmul(hidden, weights_hidden) + biases_hidden
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels))
  
  # L2 regularization
  regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) +
                 tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden))
  loss += 7e-4 * regularizers 
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)
  valid_prediction = tf.nn.softmax(
    tf.matmul( tf.nn.relu (tf.matmul(tf_valid_dataset, weights) + biases), weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu (tf.matmul(tf_test_dataset, weights) + biases), weights_hidden) + biases_hidden)

In [12]:
num_steps = 7001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 597.442749
Minibatch accuracy: 9.4%
Validation accuracy: 31.2%
Minibatch loss at step 500: 162.169006
Minibatch accuracy: 85.9%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 113.033478
Minibatch accuracy: 82.8%
Validation accuracy: 79.6%
Minibatch loss at step 1500: 77.837059
Minibatch accuracy: 86.7%
Validation accuracy: 82.5%
Minibatch loss at step 2000: 53.882259
Minibatch accuracy: 85.2%
Validation accuracy: 82.5%
Minibatch loss at step 2500: 37.890965
Minibatch accuracy: 83.6%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 26.715296
Minibatch accuracy: 89.8%
Validation accuracy: 85.3%
Minibatch loss at step 3500: 18.854862
Minibatch accuracy: 86.7%
Validation accuracy: 86.1%
Minibatch loss at step 4000: 13.420817
Minibatch accuracy: 86.7%
Validation accuracy: 87.2%
Minibatch loss at step 4500: 9.470796
Minibatch accuracy: 86.7%
Validation accuracy: 87.2%
Minibatch loss at step 5000: 6.903327
Minibatch accuracy: 88.3%
Validat

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [16]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases = tf.Variable(tf.zeros([hidden_size]))
    
  weights_hidden = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  out = tf.matmul(hidden, weights_hidden) + biases_hidden
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels))
  
  # L2 regularization
  regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) +
                 tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden))
  loss += 5e-4 * regularizers 
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)
  valid_prediction = tf.nn.softmax(
    tf.matmul( tf.nn.relu (tf.matmul(tf_valid_dataset, weights) + biases), weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu (tf.matmul(tf_test_dataset, weights) + biases), weights_hidden) + biases_hidden)

In [17]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (batch_size * 20)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 406.773193
Minibatch accuracy: 17.2%
Validation accuracy: 31.5%
Minibatch loss at step 500: 131.716965
Minibatch accuracy: 98.4%
Validation accuracy: 78.2%
Minibatch loss at step 1000: 98.810600
Minibatch accuracy: 99.2%
Validation accuracy: 77.2%
Minibatch loss at step 1500: 78.485199
Minibatch accuracy: 99.2%
Validation accuracy: 79.2%
Minibatch loss at step 2000: 58.640339
Minibatch accuracy: 99.2%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 46.123795
Minibatch accuracy: 99.2%
Validation accuracy: 79.2%
Minibatch loss at step 3000: 35.284210
Minibatch accuracy: 99.2%
Validation accuracy: 78.0%
Test accuracy: 84.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [24]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases = tf.Variable(tf.zeros([hidden_size]))
    
  weights_hidden = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  hidden = tf.nn.dropout(hidden, 0.5) #Add radom drop out rate :)
  out = tf.matmul(hidden, weights_hidden) + biases_hidden
     
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels))  
    
  # L2 regularization
  regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) +
                 tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden))
  loss += 7e-4 * regularizers 
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)
  valid_prediction = tf.nn.softmax(
    tf.matmul( tf.nn.relu (tf.matmul(tf_valid_dataset, weights) + biases), weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu (tf.matmul(tf_test_dataset, weights) + biases), weights_hidden) + biases_hidden)

In [23]:
num_steps = 7001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 682.859131
Minibatch accuracy: 9.4%
Validation accuracy: 38.5%
Minibatch loss at step 500: 170.052475
Minibatch accuracy: 76.6%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 118.402161
Minibatch accuracy: 72.7%
Validation accuracy: 79.8%
Minibatch loss at step 1500: 80.917099
Minibatch accuracy: 78.1%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 55.085701
Minibatch accuracy: 76.6%
Validation accuracy: 81.5%
Minibatch loss at step 2500: 38.626987
Minibatch accuracy: 82.0%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 27.483019
Minibatch accuracy: 78.9%
Validation accuracy: 83.9%
Minibatch loss at step 3500: 19.234606
Minibatch accuracy: 80.5%
Validation accuracy: 84.8%
Minibatch loss at step 4000: 13.713788
Minibatch accuracy: 84.4%
Validation accuracy: 85.5%
Minibatch loss at step 4500: 9.624067
Minibatch accuracy: 81.2%
Validation accuracy: 86.0%
Minibatch loss at step 5000: 7.080137
Minibatch accuracy: 83.6%
Validat

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (batch_size * 20)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 741.815186
Minibatch accuracy: 7.0%
Validation accuracy: 30.8%
Minibatch loss at step 500: 167.189026
Minibatch accuracy: 98.4%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 118.380417
Minibatch accuracy: 96.9%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 83.329185
Minibatch accuracy: 97.7%
Validation accuracy: 80.7%
Minibatch loss at step 2000: 59.684700
Minibatch accuracy: 99.2%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 43.207817
Minibatch accuracy: 99.2%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 28.574800
Minibatch accuracy: 97.7%
Validation accuracy: 81.0%
Test accuracy: 88.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---

We gonna add learning rate decay

In [38]:
batch_size = 128
hidden_size = 1024
train_size = train_labels.shape[0]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases = tf.Variable(tf.zeros([hidden_size]))
    
  weights_hidden = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  hidden = tf.nn.dropout(hidden, 0.5) #Add radom drop out rate :)
  out = tf.matmul(hidden, weights_hidden) + biases_hidden
     
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels))  
    
  # L2 regularization
  regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases) +
                 tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden))
  loss += 5e-4 * regularizers 
    
  # Optimizer. https://www.tensorflow.org/versions/r0.10/api_docs/python/train.html#exponential_decay
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(
      0.6,                 # Base learning rate.
      global_step,         # Current index into the dataset.
      900,               # Decay step.
      0.87,                # Decay rate.
      staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)
  valid_prediction = tf.nn.softmax(
    tf.matmul( tf.nn.relu (tf.matmul(tf_valid_dataset, weights) + biases), weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu (tf.matmul(tf_test_dataset, weights) + biases), weights_hidden) + biases_hidden)

In [39]:
num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 695.040161
Minibatch accuracy: 7.8%
Validation accuracy: 33.0%
Minibatch loss at step 500: 134.428268
Minibatch accuracy: 74.2%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 94.235390
Minibatch accuracy: 71.1%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 75.832657
Minibatch accuracy: 73.4%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 55.702888
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 44.368149
Minibatch accuracy: 78.1%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 35.523048
Minibatch accuracy: 76.6%
Validation accuracy: 82.9%
Minibatch loss at step 3500: 28.842525
Minibatch accuracy: 78.9%
Validation accuracy: 83.7%
Minibatch loss at step 4000: 23.612862
Minibatch accuracy: 85.2%
Validation accuracy: 84.5%
Minibatch loss at step 4500: 19.918560
Minibatch accuracy: 82.0%
Validation accuracy: 85.0%
Minibatch loss at step 5000: 17.777830
Minibatch accuracy: 82.8%
Valida

2 layer DNN. Change the name of weights and biases a bit :(. 

No dropout :)

In [58]:
batch_size = 128
hidden_size_1 = 1024
hidden_size_2 = 512
train_size = train_labels.shape[0]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size_1], stddev=np.sqrt(2.0 / (image_size * image_size)))) 
  biases_hidden_1 = tf.Variable(tf.zeros([hidden_size_1]))
    
  weights_hidden_2 = tf.Variable(
    tf.truncated_normal([hidden_size_1, hidden_size_2], stddev=np.sqrt(2.0 / (hidden_size_1))))
  biases_hidden_2 = tf.Variable(tf.zeros([hidden_size_2]))
    
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_size_2, num_labels], stddev=np.sqrt(2.0 / (hidden_size_2))))
  biases_out = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_1) + biases_hidden_1)
  #hidden_1 = tf.nn.dropout(hidden_1, 0.5) #Add radom drop out rate :)

  hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weights_hidden_2) + biases_hidden_2)
  #hidden_2 = tf.nn.dropout(hidden_2, 0.5) #Add drop out     

  out = tf.matmul(hidden_2, weights_out) + biases_out
     
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels))  
    
  # L2 regularization
  regularizers = (tf.nn.l2_loss(weights_hidden_1) + tf.nn.l2_loss(biases_hidden_1) +
                 tf.nn.l2_loss(weights_hidden_2) + tf.nn.l2_loss(biases_hidden_2) + 
                 tf.nn.l2_loss(weights_out) + tf.nn.l2_loss(biases_out))
  loss += 6e-4 * regularizers 
    
  # Optimizer. https://www.tensorflow.org/versions/r0.10/api_docs/python/train.html#exponential_decay
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(
      0.5,                 # Base learning rate.
      global_step,         # Current index into the dataset.
      1000,               # Decay step.
      0.94,                # Decay rate.
      staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)

  layer1_valid = tf.nn.relu (tf.matmul(tf_valid_dataset, weights_hidden_1) + biases_hidden_1)
  layer2_valid = tf.nn.relu (tf.matmul(layer1_valid, weights_hidden_2) + biases_hidden_2)
  valid_prediction = tf.nn.softmax( tf.matmul(layer2_valid, weights_out) + biases_out )
        
  layer1_test = tf.nn.relu (tf.matmul(tf_test_dataset, weights_hidden_1) + biases_hidden_1)
  layer2_test = tf.nn.relu (tf.matmul(layer1_test, weights_hidden_2) + biases_hidden_2)
  test_prediction = tf.nn.softmax( tf.matmul(layer2_test, weights_out) + biases_out )

In [59]:
num_steps = 11001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.966157
Minibatch accuracy: 6.2%
Validation accuracy: 30.3%
Minibatch loss at step 500: 0.873491
Minibatch accuracy: 89.8%
Validation accuracy: 86.3%
Minibatch loss at step 1000: 0.845579
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 1500: 0.686147
Minibatch accuracy: 90.6%
Validation accuracy: 87.3%
Minibatch loss at step 2000: 0.695579
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 2500: 0.556346
Minibatch accuracy: 90.6%
Validation accuracy: 88.4%
Minibatch loss at step 3000: 0.542302
Minibatch accuracy: 91.4%
Validation accuracy: 89.1%
Minibatch loss at step 3500: 0.579689
Minibatch accuracy: 85.9%
Validation accuracy: 89.1%
Minibatch loss at step 4000: 0.480518
Minibatch accuracy: 86.7%
Validation accuracy: 89.3%
Minibatch loss at step 4500: 0.444787
Minibatch accuracy: 91.4%
Validation accuracy: 89.5%
Minibatch loss at step 5000: 0.524495
Minibatch accuracy: 87.5%
Validation accuracy

With Dropout 2 hidden layers

In [64]:
batch_size = 128
hidden_size_1 = 1024
hidden_size_2 = 512
train_size = train_labels.shape[0]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size_1], stddev=np.sqrt(2.0 / (image_size * image_size)))) 
  biases_hidden_1 = tf.Variable(tf.zeros([hidden_size_1]))
    
  weights_hidden_2 = tf.Variable(
    tf.truncated_normal([hidden_size_1, hidden_size_2], stddev=np.sqrt(2.0 / (hidden_size_1))))
  biases_hidden_2 = tf.Variable(tf.zeros([hidden_size_2]))
    
  weights_out = tf.Variable(
    tf.truncated_normal([hidden_size_2, num_labels], stddev=np.sqrt(2.0 / (hidden_size_2))))
  biases_out = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_1) + biases_hidden_1)
  hidden_1 = tf.nn.dropout(hidden_1, 0.7) #Add radom drop out rate :)

  hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weights_hidden_2) + biases_hidden_2)
  hidden_2 = tf.nn.dropout(hidden_2, 0.7) #Add drop out     

  out = tf.matmul(hidden_2, weights_out) + biases_out
     
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels))  
    
  # L2 regularization
  regularizers = (tf.nn.l2_loss(weights_hidden_1) + tf.nn.l2_loss(biases_hidden_1) +
                 tf.nn.l2_loss(weights_hidden_2) + tf.nn.l2_loss(biases_hidden_2) + 
                 tf.nn.l2_loss(weights_out) + tf.nn.l2_loss(biases_out))
  loss += 5e-4 * regularizers 
    
  # Optimizer. https://www.tensorflow.org/versions/r0.10/api_docs/python/train.html#exponential_decay
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(
      0.5,                 # Base learning rate.
      global_step,         # Current index into the dataset.
      1000,               # Decay step.
      0.9,                # Decay rate.
      staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)

  layer1_valid = tf.nn.relu (tf.matmul(tf_valid_dataset, weights_hidden_1) + biases_hidden_1)
  layer2_valid = tf.nn.relu (tf.matmul(layer1_valid, weights_hidden_2) + biases_hidden_2)
  valid_prediction = tf.nn.softmax( tf.matmul(layer2_valid, weights_out) + biases_out )
        
  layer1_test = tf.nn.relu (tf.matmul(tf_test_dataset, weights_hidden_1) + biases_hidden_1)
  layer2_test = tf.nn.relu (tf.matmul(layer1_test, weights_hidden_2) + biases_hidden_2)
  test_prediction = tf.nn.softmax( tf.matmul(layer2_test, weights_out) + biases_out )

In [65]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):

    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.236030
Minibatch accuracy: 7.8%
Validation accuracy: 32.0%
Minibatch loss at step 500: 0.901588
Minibatch accuracy: 89.1%
Validation accuracy: 85.8%
Minibatch loss at step 1000: 0.922471
Minibatch accuracy: 88.3%
Validation accuracy: 86.7%
Minibatch loss at step 1500: 0.780074
Minibatch accuracy: 89.1%
Validation accuracy: 87.4%
Minibatch loss at step 2000: 0.718712
Minibatch accuracy: 85.9%
Validation accuracy: 88.1%
Minibatch loss at step 2500: 0.624522
Minibatch accuracy: 87.5%
Validation accuracy: 88.4%
Minibatch loss at step 3000: 0.634220
Minibatch accuracy: 88.3%
Validation accuracy: 88.8%
Minibatch loss at step 3500: 0.679474
Minibatch accuracy: 85.2%
Validation accuracy: 89.0%
Minibatch loss at step 4000: 0.556101
Minibatch accuracy: 85.9%
Validation accuracy: 89.2%
Minibatch loss at step 4500: 0.545117
Minibatch accuracy: 88.3%
Validation accuracy: 89.2%
Minibatch loss at step 5000: 0.588525
Minibatch accuracy: 86.7%
Validation accuracy

In [74]:
# No dropout

batch_size = 128
hidden_size_1 = 1024
hidden_size_2 = 512
hidden_size_3 = 256
train_size = train_labels.shape[0]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size_1], stddev=np.sqrt(2.0 / (image_size * image_size)))) 
  biases_hidden_1 = tf.Variable(tf.zeros([hidden_size_1]))
    
  weights_hidden_2 = tf.Variable(
    tf.truncated_normal([hidden_size_1, hidden_size_2], stddev=np.sqrt(2.0 / (hidden_size_1))))
  biases_hidden_2 = tf.Variable(tf.zeros([hidden_size_2]))

  weights_hidden_3 = tf.Variable(
    tf.truncated_normal([hidden_size_2, hidden_size_3], stddev=np.sqrt(2.0 / (hidden_size_2))))
  biases_hidden_3 = tf.Variable(tf.zeros([hidden_size_3]))

  weights_out = tf.Variable(
    tf.truncated_normal([hidden_size_3, num_labels], stddev=np.sqrt(2.0 / (hidden_size_3))))
  biases_out = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_1) + biases_hidden_1)
  #hidden_1 = tf.nn.dropout(hidden_1, 0.7) #Add radom drop out rate :)

  hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weights_hidden_2) + biases_hidden_2)
  #hidden_2 = tf.nn.dropout(hidden_2, 0.7) #Add drop out

  hidden_3 = tf.nn.relu(tf.matmul(hidden_2, weights_hidden_3) + biases_hidden_3)
  #hidden_3 = tf.nn.dropout(hidden_3, 0.7) #Add drop out  

  out = tf.matmul(hidden_3, weights_out) + biases_out
     
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels))  
    
  # L2 regularization
  regularizers = (tf.nn.l2_loss(weights_hidden_1) + tf.nn.l2_loss(biases_hidden_1) +
                 tf.nn.l2_loss(weights_hidden_2) + tf.nn.l2_loss(biases_hidden_2) +
                 tf.nn.l2_loss(weights_hidden_3) + tf.nn.l2_loss(biases_hidden_3) +
                 tf.nn.l2_loss(weights_out) + tf.nn.l2_loss(biases_out))
  loss += 5e-4 * regularizers 
    
  # Optimizer. https://www.tensorflow.org/versions/r0.10/api_docs/python/train.html#exponential_decay
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(
      0.5,                 # Base learning rate.
      global_step,         # Current index into the dataset.
      1000,                # Decay step.
      0.9,                # Decay rate.
      staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)

  layer1_valid = tf.nn.relu (tf.matmul(tf_valid_dataset, weights_hidden_1) + biases_hidden_1)
  layer2_valid = tf.nn.relu (tf.matmul(layer1_valid, weights_hidden_2) + biases_hidden_2)
  layer3_valid = tf.nn.relu (tf.matmul(layer2_valid, weights_hidden_3) + biases_hidden_3)
  valid_prediction = tf.nn.softmax( tf.matmul(layer3_valid, weights_out) + biases_out )
        
  layer1_test = tf.nn.relu (tf.matmul(tf_test_dataset, weights_hidden_1) + biases_hidden_1)
  layer2_test = tf.nn.relu (tf.matmul(layer1_test, weights_hidden_2) + biases_hidden_2)
  layer3_test = tf.nn.relu (tf.matmul(layer2_test, weights_hidden_3) + biases_hidden_3)
  test_prediction = tf.nn.softmax( tf.matmul(layer3_test, weights_out) + biases_out )

In [75]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):

    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.149536
Minibatch accuracy: 9.4%
Validation accuracy: 28.2%
Minibatch loss at step 500: 0.932822
Minibatch accuracy: 90.6%
Validation accuracy: 86.2%
Minibatch loss at step 1000: 0.918769
Minibatch accuracy: 89.8%
Validation accuracy: 87.3%
Minibatch loss at step 1500: 0.751587
Minibatch accuracy: 89.8%
Validation accuracy: 87.1%
Minibatch loss at step 2000: 0.737911
Minibatch accuracy: 86.7%
Validation accuracy: 89.2%
Minibatch loss at step 2500: 0.588397
Minibatch accuracy: 90.6%
Validation accuracy: 88.6%
Minibatch loss at step 3000: 0.573090
Minibatch accuracy: 89.8%
Validation accuracy: 89.2%
Minibatch loss at step 3500: 0.598429
Minibatch accuracy: 85.2%
Validation accuracy: 88.8%
Minibatch loss at step 4000: 0.522290
Minibatch accuracy: 87.5%
Validation accuracy: 89.6%
Minibatch loss at step 4500: 0.482182
Minibatch accuracy: 90.6%
Validation accuracy: 89.7%
Minibatch loss at step 5000: 0.534507
Minibatch accuracy: 89.8%
Validation accuracy

In [17]:
# 3 hidden layers with drop out, consolidated version
batch_size = 128
hidden_size_1 = 1024
hidden_size_2 = 512
hidden_size_3 = 256

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_hidden_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size_1], stddev=np.sqrt(2.0 / (image_size * image_size)))) 
  biases_hidden_1 = tf.Variable(tf.zeros([hidden_size_1]))
    
  weights_hidden_2 = tf.Variable(
    tf.truncated_normal([hidden_size_1, hidden_size_2], stddev=np.sqrt(2.0 / (hidden_size_1))))
  biases_hidden_2 = tf.Variable(tf.zeros([hidden_size_2]))

  weights_hidden_3 = tf.Variable(
    tf.truncated_normal([hidden_size_2, hidden_size_3], stddev=np.sqrt(2.0 / (hidden_size_2))))
  biases_hidden_3 = tf.Variable(tf.zeros([hidden_size_3]))

  weights_out = tf.Variable(
    tf.truncated_normal([hidden_size_3, num_labels], stddev=np.sqrt(2.0 / (hidden_size_3))))
  biases_out = tf.Variable(tf.zeros([num_labels]))
    
  #Function to define the model inside :)  
  def model(data, train=False):    
    #Layer 1
    hidden_1 = tf.nn.relu(tf.matmul(data, weights_hidden_1) + biases_hidden_1)
    if (train):
      hidden_1 = tf.nn.dropout(hidden_1, 0.7) #Add radom drop out rate :)
    
    #Layer 2
    hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weights_hidden_2) + biases_hidden_2)
    if (train):
      hidden_2 = tf.nn.dropout(hidden_2, 0.7) #Add drop out
    
    #Layer3
    hidden_3 = tf.nn.relu(tf.matmul(hidden_2, weights_hidden_3) + biases_hidden_3)
    if (train):
      hidden_3 = tf.nn.dropout(hidden_3, 0.7) #Add drop out  

    return tf.matmul(hidden_3, weights_out) + biases_out
     
  out = model(tf_train_dataset, train=True)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels))  
    
  # L2 regularization
  regularizers = (tf.nn.l2_loss(weights_hidden_1) + tf.nn.l2_loss(biases_hidden_1) +
                 tf.nn.l2_loss(weights_hidden_2) + tf.nn.l2_loss(biases_hidden_2) +
                 tf.nn.l2_loss(weights_hidden_3) + tf.nn.l2_loss(biases_hidden_3) +
                 tf.nn.l2_loss(weights_out) + tf.nn.l2_loss(biases_out))
  loss += 3e-4 * regularizers 
    
  # Optimizer. https://www.tensorflow.org/versions/r0.10/api_docs/python/train.html#exponential_decay
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(
      0.5,                 # Base learning rate.
      global_step,         # Current index into the dataset.
      1000,                # Decay step.
      0.9,                # Decay rate.
      staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)

  valid_prediction = model(tf_valid_dataset)        
  test_prediction = model(tf_test_dataset)

In [18]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):

    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.833661
Minibatch accuracy: 8.6%
Validation accuracy: 23.8%
Minibatch loss at step 500: 0.853863
Minibatch accuracy: 88.3%
Validation accuracy: 85.4%
Minibatch loss at step 1000: 0.893638
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Minibatch loss at step 1500: 0.714617
Minibatch accuracy: 89.1%
Validation accuracy: 87.3%
Minibatch loss at step 2000: 0.717446
Minibatch accuracy: 85.9%
Validation accuracy: 88.3%
Minibatch loss at step 2500: 0.631955
Minibatch accuracy: 89.1%
Validation accuracy: 88.4%
Minibatch loss at step 3000: 0.640312
Minibatch accuracy: 85.9%
Validation accuracy: 88.7%
Minibatch loss at step 3500: 0.775347
Minibatch accuracy: 82.8%
Validation accuracy: 88.6%
Minibatch loss at step 4000: 0.589336
Minibatch accuracy: 86.7%
Validation accuracy: 89.0%
Minibatch loss at step 4500: 0.551126
Minibatch accuracy: 89.8%
Validation accuracy: 89.3%
Minibatch loss at step 5000: 0.640618
Minibatch accuracy: 85.9%
Validation accuracy